In [1]:
import os
os.environ["DJANGO_SETTINGS_MODULE"] = "ClusterCast.settings"
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from django.db.models.functions import Now
from asgiref.sync import sync_to_async
from tensorflow.keras.layers import Input, LSTM, Dropout, RepeatVector, TimeDistributed, Dense, Masking
from tensorflow.keras.models import Model
import ClusterPipeline.models.RNNModels as rnn
from tensorflow.keras.callbacks import EarlyStopping

import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.RNNModels as rnn 

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f4d733f92d0>
Traceback (most recent call last):
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/usr/remote/apps/anaconda/2023.03-acet116/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
2024-01-11 01:24:05.719351: I tensorflow/core/pl

In [3]:
@sync_to_async
def get_all_objects(features,steps = None):
    # Force the query to execute and load all results into memory
    params = list(cp.StockClusterGroupParams.objects.all())
    matching_groups = []
    for param in params:
        if steps is None:
            if param.cluster_features == features:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()

        else: 
            if param.cluster_features == features and param.n_steps == steps:
                group = cp.StockClusterGroup.objects.get(pk=param.pk)
                matching_groups.append(group)
                group.load_saved_group()
                print(group.group_params.tickers)
    
    return matching_groups


cluster_features = ["close", "bb_low", "bb_high"]
steps = 20

async def create_cluster_group_params(cluster_features, steps=20):

    cluster_groups = await get_all_objects(features=cluster_features,steps=steps)
    return cluster_groups

# Run the async function

matching_groups = await create_cluster_group_params(cluster_features, steps=steps)

Creating and Processing Dataset


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Scaling Quant Min Max Features
Quant Min Max Features Scaled
Dataset Preprocessing Complete
Creating Sequences
Scaling Sequences
Scaling Sequences Complete
['SPY']
Creating and Processing Dataset
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Scaling Quant Min Max Features
Quant Min Max Features Scaled
Dataset Preprocessing Complete
Creating Sequences
Scaling Sequences
Scaling Sequences Complete
['F']


In [4]:
@sync_to_async
def get_all_clusters(groups):
    clusters = []
    features = []
    for group in groups:
        clusters += group.clusters

    for cluster in clusters:
        models = rnn.RNNModel.objects.filter(cluster=cluster)
        for model in models: 
            if model:
                features += (model.model_features)

    features = list(set(features))
    return clusters, features

clusters, all_features = await get_all_clusters(matching_groups)



In [5]:
test_cluster = clusters[8]

In [6]:
import random
features = random.sample(all_features, 50)

feature_indices = [test_cluster.X_feature_dict[feature] for feature in features] 
test_cluster.X_train_filtered = test_cluster.X_train[:,:,feature_indices]
test_cluster.X_test_filtered = test_cluster.X_test[:,:,feature_indices]


In [7]:
from keras.layers import Input, LSTM, Dense, Dropout, Multiply, Permute, RepeatVector, TimeDistributed, Flatten, Activation, Lambda
from keras.models import Model
from keras.optimizers import Adam

def create_model_with_attention_and_teacher_forcing(input_shape, output_steps, latent_dim=6):
    # Encoder
    encoder_input = Input(shape=(None, input_shape))
    encoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True)(encoder_input)
    encoder_lstm2 = LSTM(units=latent_dim, activation='tanh', return_sequences=True)(encoder_lstm1)
    encoder_output = Dropout(0.2)(encoder_lstm2)

    # Attention Mechanism
    attention = Dense(1, activation='tanh')(encoder_output)
    attention = Flatten()(attention)
    attention_weights = Activation('softmax')(attention)
    context = Multiply()([encoder_output, Permute([2, 1])(RepeatVector(latent_dim)(attention_weights))])
    context_vector = Lambda(lambda x: x[:, -1, :])(context)  # Use the last context vector for initial state

    # Adjust the dimensionality of the context vector
    context_dense = Dense(75)(context_vector)  # Transform to match decoder LSTM units

    # Decoder
    decoder_input = Input(shape=(output_steps, 1))  # Adjust the shape as necessary
    decoder_lstm = LSTM(units=75, activation='tanh', return_sequences=True)(decoder_input, initial_state=[context_dense, context_dense])
    decoder_output = Dropout(0.2)(decoder_lstm)

    # TimeDistributed layer for output
    time_distributed_output = TimeDistributed(Dense(1))(decoder_output)

    # Model
    model = Model(inputs=[encoder_input, decoder_input], outputs=time_distributed_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

    return model



In [8]:
def create_model(input_shape, latent_dim=6):
    # Input layer
    input_layer = Input(shape=(None, input_shape))

    # masking_layer = Masking(mask_value=0.0, name='masking_layer')(input_layer)

    # Encoder

    encoder_lstm2 = LSTM(units=50, activation='tanh', return_sequences=True, name='encoder_lstm_2_restore')(input_layer)
    encoder_dropout2 = Dropout(0.2, name='encoder_dropout_2_restore')(encoder_lstm2)

    encoder_lstm3 = LSTM(units=100, activation='tanh', return_sequences=False, name='encoder_lstm_3_restore')(encoder_dropout2)
    encoder_dropout3 = Dropout(0.2, name='encoder_dropout_3_restore')(encoder_lstm3)

    # encoder_lstm4 = LSTM(units=50, activation='tanh', return_sequences=False, name='encoder_lstm_4_restore')(encoder_dropout3)
    # encoder_dropout4 = Dropout(0.2, name='encoder_dropout_4_restore')(encoder_lstm4)

    # Repeat Vector
    repeat_vector = RepeatVector(latent_dim, name='repeat_vector')(encoder_dropout3)

    # Decoder
    decoder_lstm1 = LSTM(units=50, activation='tanh', return_sequences=True, name='decoder_lstm_1_restore')(repeat_vector)
    decoder_dropout1 = Dropout(0.2, name='decoder_dropout_1_restore')(decoder_lstm1)

    decoder_lstm2 = LSTM(units=25, activation='tanh', return_sequences=True, name='decoder_lstm_2_restore')(decoder_dropout1)
    decoder_dropout2 = Dropout(0.2, name='decoder_dropout_2_restore')(decoder_lstm2)

    time_distributed_output = TimeDistributed(Dense(1), name='time_distributed_output')(decoder_dropout2)


    # Create the model
    model_lstm = Model(inputs=input_layer, outputs=time_distributed_output)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model_lstm.compile(optimizer=optimizer, loss="mse")

    return model_lstm


In [9]:
import numpy as np
X_train = test_cluster.X_train_filtered
y_train = test_cluster.y_train
X_test = test_cluster.X_test_filtered
y_test = test_cluster.y_test

# Assuming y_train is your training data with shape (elements, output_steps)
y_train_shifted = np.zeros_like(y_train)

# Shift y_train by one time step
y_train_shifted[:, 1:] = y_train[:, :-1]

# Adding an extra dimension for features (assuming single feature per step)
decoder_input_train = np.expand_dims(y_train_shifted, -1)

start_token = 0

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(decoder_input_train.shape)



(188, 20, 50)
(188, 6)
(43, 20, 50)
(43, 6)
(188, 6, 1)


In [10]:
regular_model = create_model(X_train.shape[2], latent_dim=6)
attention_model = create_model_with_attention_and_teacher_forcing(X_train.shape[2], 6,latent_dim=6)

2024-01-11 01:26:21.515402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-11 01:26:21.555631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-11 01:26:21.555783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
import pandas as pd 
import numpy as np
def eval_model(X_test, y_test, model,cluster):

    predicted_y = model.predict(X_test)
    predicted_y = np.squeeze(predicted_y, axis=-1)

    num_days = predicted_y.shape[1]  # Assuming this is the number of days
    results = pd.DataFrame(predicted_y, columns=[f'predicted_{i+1}' for i in range(num_days)])

    for i in range(num_days):
        results[f'real_{i+1}'] = y_test[:, i]

    # Generate output string with accuracies
    output_string = f"Cluster Number: {cluster.label}\n"
    for i in range(num_days):
        same_day = ((results[f'predicted_{i+1}'] > 0) & (results[f'real_{i+1}'] > 0)) | \
                ((results[f'predicted_{i+1}'] < 0) & (results[f'real_{i+1}'] < 0))
        accuracy = round(same_day.mean() * 100,2)

        output_string += (
            f"Accuracy{i+1}D {accuracy}% "
            f"PredictedRet: {results[f'predicted_{i+1}'].mean()} "
            f"ActRet: {results[f'real_{i+1}'].mean()}\n"
        )
    
    output_string += f"Train set length: {len(X_train)} Test set length: {len(y_test)}\n"

    return output_string, results

In [ ]:
optimizer = Adam(learning_rate=0.0001)
regular_model.compile(optimizer=optimizer, loss='mse')

# Set up early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=35, restore_best_weights=True)

# Fine-tune the model using your smaller dataset
# Assume you have 'small_train_data', 'small_train_labels', 'small_val_data', and 'small_val_labels'
history = regular_model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=250,
                    batch_size=16,
                    callbacks=[early_stopping])

# Evaluate the model performance
val_loss = regular_model.evaluate(X_test, y_test)
regularAccuracy,regularResults = eval_model(X_test, y_test, regular_model,test_cluster)
clear_session()
del regular_model
print(f'Validation loss: {val_loss}')

In [12]:
# optimizer = Adam(learning_rate=0.0001)
# attention_model.compile(optimizer=optimizer, loss='mse')

# Set up early stopping to avoid overfitting
# Compile the model
attention_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = attention_model.fit([X_train, decoder_input_train], y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_split=0.2)

# Evaluate the model performance
val_loss = attention_model.evaluate(X_test, y_test)
print(f'Validation loss: {val_loss}')
attentionAccuracy,attentionResults = eval_model(X_test, y_test, attention_model,test_cluster)

Epoch 1/100


2024-01-11 01:26:27.212320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-11 01:26:27.213461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-11 01:26:27.214298: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

InternalError: Graph execution error:

Failed to call ThenRnnBackward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 1, 75, 1, 6, 32, 75] 
	 [[{{node gradients/CudnnRNN_grad/CudnnRNNBackprop}}]]
	 [[Adam/gradients/PartitionedCall]] [Op:__inference_train_function_10383]

In [ ]:
import plotly.graph_objects as go
def visualize_future_distribution(results):
    '''
    Create stacked box and whisker plots for the predicted and real values
    '''

    fig = go.Figure()
    print(results.shape)

    for i in range(6):

        fig.add_trace(go.Box(y=results[f'predicted_{i+1}'], name=f'Predicted {i}')) 
        fig.add_trace(go.Box(y=results[f'real_{i+1}'], name=f'Real {i}'))

    fig.update_layout(
        title='Future Performance of Cluster',
        xaxis_title='Steps in future',
        yaxis_title='Cumulative Percent Change'
    ) 

    return fig

In [ ]:
from plotly.subplots import make_subplots
bench_fig = visualize_future_distribution(regularResults)
tuned_fig = visualize_future_distribution(attentionResults)
fig = make_subplots(rows=1, cols=2)

for trace in bench_fig.data:
    fig.add_trace(trace, row=1, col=1)

for trace in tuned_fig.data:
    fig.add_trace(trace, row=1, col=2)

fig.show()

In [ ]:
print(regularAccuracy)
print(attentionAccuracy)